In [61]:
import numpy as np
import os

In [62]:
# Function to read obj file and compute the number of vertices
def read_obj_file(file_path):
    vertices = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                components = line.split()[1:]
                vertex = [float(comp) for comp in components]
                vertices.append(vertex)

    return np.array(vertices)

# Sample
sample_cad_model = read_obj_file("/home/hrishi/Programming/Python/CAD-DR/abc-dataset/00030037_0ef34aa1b15748a5b4ad7c0e_trimesh_027.obj")
print(sample_cad_model)
print(sample_cad_model.shape)

[[  8.6       -0.        -6.25    ]
 [ -8.6       -0.        -6.25    ]
 [  8.6      -10.        -6.25    ]
 ...
 [ -2.526908  -5.088639   6.25    ]
 [  3.918603  -4.431913   6.25    ]
 [ -3.283615  -5.150998   6.25    ]]
(22323, 3)


In [63]:
# Finding the number of vertices of each model in the dataset
max_vertices = 0
for filename in os.listdir("abc-dataset"):
    vertices = len(read_obj_file("abc-dataset/" + str(filename)))
    # print(vertices) - can be used if we wish to see number of vertices of each model
    if (vertices > max_vertices):
        max_vertices = vertices

In [64]:
# Maximum number of vertices
print(max_vertices)

635732
